In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd

In [2]:
class Sampling(layers.Layer):
    """Uses (z_mean, z_log_var) to sample z, the vector encoding a digit."""

    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon

In [3]:
latent_dim = 2

encoder_inputs = keras.Input(shape=(17, 1))
x = layers.Conv1D(8,3, activation='relu', padding='same',dilation_rate=2)(encoder_inputs)
# x = ZeroPadding1D((2,1))(x)
x1 = layers.MaxPooling1D(1)(x)
x2 = layers.Conv1D(4,3, activation='relu', padding='same',dilation_rate=2)(x1)
x3 = layers.MaxPooling1D(1)(x2)
x4 = layers.AveragePooling1D()(x3)
flat = layers.Flatten()(x4)
encoder = layers.Dense(2)(flat)
x = encoder
z_mean = layers.Dense(latent_dim, name="z_mean")(x)
z_log_var = layers.Dense(latent_dim, name="z_log_var")(x)
z = Sampling()([z_mean, z_log_var])
encoder = keras.Model(encoder_inputs, [z_mean, z_log_var, z], name="encoder")
encoder.summary()


Model: "encoder"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 17, 1)]      0           []                               
                                                                                                  
 conv1d (Conv1D)                (None, 17, 8)        32          ['input_1[0][0]']                
                                                                                                  
 max_pooling1d (MaxPooling1D)   (None, 17, 8)        0           ['conv1d[0][0]']                 
                                                                                                  
 conv1d_1 (Conv1D)              (None, 17, 4)        100         ['max_pooling1d[0][0]']          
                                                                                            

In [4]:
latent_inputs = keras.Input(shape=(latent_dim,))
#d = layers.Flatten()(latent_inputs)
d1 = layers.Dense(9)(latent_inputs)
#d2 = layers.Reshape((117,4))(d1)
d2 = layers.Reshape((9,1))(d1)
d3 = layers.Conv1D(4,1,strides=1, activation='relu', padding='same')(d2)
d4 = layers.UpSampling1D()(d3)
d5 = layers.Conv1D(8,1,strides=1, activation='relu', padding='same')(d4)
#d6 = layers.UpSampling1D(2)(d5)
#d7 = layers.UpSampling1D(2)(d6)
d8 = layers.Conv1D(1,2, strides=1, activation='sigmoid', padding='same')(d5)
decoded = layers.Cropping1D(cropping=(1,0))(d8) # this is the added step
#decoded = d8

decoder = keras.Model(latent_inputs, decoded, name="decoder")
decoder.summary()

Model: "decoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 2)]               0         
                                                                 
 dense_1 (Dense)             (None, 9)                 27        
                                                                 
 reshape (Reshape)           (None, 9, 1)              0         
                                                                 
 conv1d_2 (Conv1D)           (None, 9, 4)              8         
                                                                 
 up_sampling1d (UpSampling1D  (None, 18, 4)            0         
 )                                                               
                                                                 
 conv1d_3 (Conv1D)           (None, 18, 8)             40        
                                                           

In [5]:
class VAE(keras.Model):
    def __init__(self, encoder, decoder, **kwargs):
        super(VAE, self).__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        self.total_loss_tracker = keras.metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker = keras.metrics.Mean(
            name="reconstruction_loss"
        )
        self.kl_loss_tracker = keras.metrics.Mean(name="kl_loss")

    @property
    def metrics(self):
        return [
            self.total_loss_tracker,
            self.reconstruction_loss_tracker,
            self.kl_loss_tracker,
        ]

    def train_step(self, data):
        with tf.GradientTape() as tape:
            z_mean, z_log_var, z = self.encoder(data)
            reconstruction = self.decoder(z)
            reconstruction_loss = tf.reduce_mean(
                tf.reduce_sum(
                    (data - reconstruction)**2, axis=(1, 2)
                )
            )
            kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
            kl_loss = tf.reduce_mean(tf.reduce_sum(kl_loss, axis=1))
            total_loss = reconstruction_loss + kl_loss
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        return {
            "loss": self.total_loss_tracker.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result(),
        }

In [6]:
(x_train) = pd.read_csv('train_dataset.csv')
(x_test) = pd.read_csv('test_dataset.csv')

#print(x_train)
#print(x_test)

In [7]:
peptides = np.concatenate([x_train, x_test], axis=0)
#mnist_digits = np.expand_dims(mnist_digits, -1).astype("float32") / 255
peptides = np.expand_dims(peptides, -1)

vae = VAE(encoder, decoder)
vae.compile(optimizer=keras.optimizers.Adam())
vae.fit(peptides, epochs=3)

Epoch 1/3
33/33 [==============================] - 8s 207ms/step - loss: 7554973.2647 - reconstruction_loss: 7400557.5000 - kl_loss: 471805.5938
Epoch 2/3
33/33 [==============================] - 7s 203ms/step - loss: 7605674.8235 - reconstruction_loss: 7401751.0000 - kl_loss: 525.1280
Epoch 3/3
33/33 [==============================] - 6s 171ms/step - loss: 7580708.6912 - reconstruction_loss: 7400849.5000 - kl_loss: 541.3251
